# Random Forest

## 1)- Importamos todas las librerías que necesitaremos para nuestro proyecto

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

## 2)- Ahora vamos a leer el csv y preparar los conjuntos de datos

In [2]:
data = pd.read_csv('Data_house_cleaned.csv')
data.head(5)

,price,bedrooms,floors,view,condition,sqft_above,sqft_basement,2,20,22,...,57,58,62,63,72,73,2.1,20.1,22.1,sqft_above_yeojohnson
0,313000.0,-0.441122,-0.022416,-0.309194,-0.667112,-0.565224,-0.672464,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.443794
1,342000.0,-0.441122,-0.951388,-0.309194,0.809652,0.119171,-0.672464,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.531898
2,420000.0,-0.441122,-0.951388,-0.309194,0.809652,-0.959621,1.482306,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.199296
3,550000.0,0.659291,-0.951388,-0.309194,0.809652,-0.797222,1.051352,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.871731
4,490000.0,-1.541536,-0.951388,-0.309194,-0.667112,-1.098820,-0.672464,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.498083


## 3)- Separación variable dependiente e independiente

In [3]:
y = data['price']

In [4]:
X = data.iloc[:,1:-1]

In [5]:
X

,bedrooms,floors,view,condition,sqft_above,sqft_basement,2,20,22,44,...,55,57,58,62,63,72,73,2.1,20.1,22.1
0,-0.441122,-0.022416,-0.309194,-0.667112,-0.565224,-0.672464,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.441122,-0.951388,-0.309194,0.809652,0.119171,-0.672464,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.441122,-0.951388,-0.309194,0.809652,-0.959621,1.482306,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.659291,-0.951388,-0.309194,0.809652,-0.797222,1.051352,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.541536,-0.951388,-0.309194,-0.667112,-1.098820,-0.672464,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3836,-0.441122,-0.951388,-0.309194,0.809652,-0.368025,-0.672464,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3837,-0.441122,0.906555,-0.309194,-0.667112,-0.426025,-0.672464,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3838,-0.441122,0.906555,-0.309194,-0.667112,1.371962,-0.672464,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3839,0.659291,-0.951388,-0.309194,-0.667112,-0.878421,1.525401,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### En este caso no usamos la columna de 'sqft_above_yeojohnson' porque estamos creando un random forest y no es necesario en este caso

## 4)- Entrenamiento y ajuste del modelo

#### En esta ocasión vamos a implementar el GridSearchCV para poder obtener el mejor conjunto de hiperparámetros para nuestro modelo

In [13]:
parameters= [{'criterion':["squared_error", "absolute_error", "friedman_mse", "poisson"],'n_estimators':[200,300,400,500],'max_depth' : [10,20,30],'min_samples_split' : [4,6,8]}]

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [10]:
regresor = RandomForestRegressor()

In [11]:
grid_search = GridSearchCV(regresor,param_grid=parameters,scoring='neg_mean_squared_error',cv = 2,n_jobs=-1)

In [14]:
grid_search = grid_search.fit(X_train,y_train)

In [16]:
grid_search.best_params_

{'criterion': 'absolute_error',
 'max_depth': 30,
 'min_samples_split': 8,
 'n_estimators': 300}

In [47]:
randomForest = RandomForestRegressor(n_estimators=300,criterion= "absolute_error",n_jobs=5,max_depth=30,min_samples_split=8)
randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)

In [18]:
(mean_absolute_error(y_test, y_pred) / data['price'].mean() )*100

20.113946032238093

#### Una vez que hemos probado los primeros parámetros, iremos ajustando el rango de los mismos para mejorar el modelo hasta que ya sufra de overfiting y empeore

In [48]:
parameters= [{'criterion':["absolute_error"],'n_estimators':[300],'max_depth' : [36,28,30],'min_samples_split' : [10,6,8], 'max_features': [10,12,14]}]

In [49]:
grid_search = GridSearchCV(regresor,param_grid=parameters,scoring='neg_mean_squared_error',cv = 4,n_jobs=-1)

In [50]:
grid_search = grid_search.fit(X_train,y_train)

In [51]:
grid_search.best_params_

{'criterion': 'absolute_error',
 'max_depth': 30,
 'max_features': 10,
 'min_samples_split': 8,
 'n_estimators': 300}

In [53]:
randomForest = RandomForestRegressor(n_estimators=300,criterion= "absolute_error",n_jobs=5,max_depth=30,min_samples_split=8,max_features=10)
randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)
(mean_absolute_error(y_test, y_pred) / data['price'].mean() )*100

19.45992725685691

In [71]:
parameters= [{'criterion':["absolute_error"],'n_estimators':[300],'max_depth' : [30,28,26],'min_samples_split' : [6,8,10],'max_features': [10,8,9]}]

In [72]:
grid_search = GridSearchCV(regresor,param_grid=parameters,scoring='r2',cv = 4,n_jobs=-1)

In [73]:
grid_search = grid_search.fit(X_train,y_train)

In [68]:
grid_search.best_params_

{'criterion': 'absolute_error',
 'max_depth': 22,
 'max_features': 6,
 'min_samples_split': 7,
 'n_estimators': 300}

In [62]:
randomForest = RandomForestRegressor(n_estimators=300,criterion= "absolute_error",n_jobs=5,max_depth=26,min_samples_split=10,max_features=8)
randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)
(mean_absolute_error(y_test, y_pred) / data['price'].mean() )*100

19.402278435623934

In [65]:
parameters= [{'criterion':["absolute_error"],'n_estimators':[300],'max_depth' : [26,24,22],'min_samples_split' : [5,10,7],'max_features' : [8,6,9]}]

In [66]:
grid_search = GridSearchCV(regresor,param_grid=parameters,scoring='r2',cv = 4,n_jobs=-1)

In [67]:
grid_search = grid_search.fit(X_train,y_train)

In [69]:
grid_search.best_params_

{'criterion': 'absolute_error',
 'max_depth': 22,
 'max_features': 6,
 'min_samples_split': 7,
 'n_estimators': 300}

In [70]:
randomForest = RandomForestRegressor(n_estimators=300,criterion= "absolute_error",n_jobs=5,max_depth=22,min_samples_split=7,max_features=6)
randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)
(mean_absolute_error(y_test, y_pred) / data['price'].mean() )*100

19.33088846549742

In [75]:
parameters= [{'criterion':["absolute_error"],'n_estimators':[300],'max_depth' : [18,20,22],'min_samples_split' : [5,8,7],'max_features' : [5,6,7]}]

In [76]:
grid_search = GridSearchCV(regresor,param_grid=parameters,scoring='r2',cv = 4,n_jobs=-1)

In [77]:
grid_search = grid_search.fit(X_train,y_train)

In [78]:
grid_search.best_params_

{'criterion': 'absolute_error',
 'max_depth': 22,
 'max_features': 5,
 'min_samples_split': 8,
 'n_estimators': 300}

In [79]:
randomForest = RandomForestRegressor(n_estimators=300,criterion= "absolute_error",n_jobs=5,max_depth=22,min_samples_split=8,max_features=5)
randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)
(mean_absolute_error(y_test, y_pred) / data['price'].mean() )*100

19.314078968775

In [88]:
parameters= [{'criterion':["absolute_error"],'n_estimators':[300],'max_depth' : [18,20,22],'min_samples_split' : [5,8,7],'max_features' : [5,4,7],'min_samples_leaf' : [2,3,4]}]

In [89]:
grid_search = GridSearchCV(regresor,param_grid=parameters,scoring='r2',cv = 4,n_jobs=-1)

In [90]:
grid_search = grid_search.fit(X_train,y_train)

In [91]:
grid_search.best_params_

{'criterion': 'absolute_error',
 'max_depth': 22,
 'max_features': 7,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 300}

In [92]:
randomForest = RandomForestRegressor(n_estimators=300,criterion= "absolute_error",n_jobs=5,max_depth=22,min_samples_split=5,max_features=7,min_samples_leaf=2)
randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)
(mean_absolute_error(y_test, y_pred) / data['price'].mean() )*100

20.348441333364967

#### Hasta ahora el mejor modelo que he obtenido usando el Grid Search ha sido el que usa 300 árboles, usa el criterio del error absoluto que va muy bien para mitigar el efecto de los valores extremos, un max_depth de 22, min_samples_split de 8 y un max_features de 5
#### El mejor resultado es de 80.69% de precisión